# ETL

Se importan las bibliotecas necesarias para el manejo de los datos, para la carga debemos tener en cuenta que los archivos se encuentran en JSON y queremos convertirlos en un Dataframe.

In [1]:
import pandas as pd
from pandas import json_normalize
import json as js
import ast as ast
import re
from textblob import TextBlob

### Carga de datos

Para el Dataset de steam_games, se pudo utilizar el pd.read para la carga de sus datos y funcionó correctamente. Los Datasets de user_reviews y user_items al contener listas anidadas en sus celdas, se tuvo que proceder con un código diferente: Se crea una lista vacía y se utiliza un bucle for para leer cada línea del archivo JSON, convertirla a un diccionario de Python y agregarla a la lista. Finalmente, la lista se convierte en un DataFrame de Pandas usando pd.DataFrame.

In [2]:
games = pd.read_json('output_steam_games.json', lines = True)

In [3]:
list_rev = []
archivo1 = r'australian_user_reviews.json'
with open(archivo1, encoding='utf-8') as file:
    for line in file.readlines():
        list_rev.append(ast.literal_eval(line))

reviews = pd.DataFrame(list_rev)

In [4]:
list_items = []
archivo2 = r'australian_users_items.json'
with open(archivo2, encoding='utf-8') as file:
    for line in file.readlines():
        list_items.append(ast.literal_eval(line))

items = pd.DataFrame(list_items)

### Desanidación de tablas

Ya tenemos las tablas cargadas, pero se puede observar que los Dataframes de Reviews e Items contienen ambos una columna con listas anidadas ('reviews' e 'items'), se procede entonces a la desanidación de estas para poder trabajar con el Dataframe de la manera más optima.

In [5]:
reviews = reviews.explode('reviews', ignore_index=True)
reviews_expanded = pd.json_normalize(reviews['reviews'])
reviews_expanded=reviews_expanded.replace('',None)
reviews = reviews.join(reviews_expanded)
reviews.drop(columns=['reviews'],inplace=True)

In [6]:
items = items.explode('items', ignore_index=True)
items_expanded = pd.json_normalize(items['items'])
items_expanded = items_expanded.replace('', None)
items = items.join(items_expanded)
items.drop(columns=['items'], inplace=True)

### Eliminación de columnas innecesarias

Después de leer todas las consultas y exigencias del proyecto a realizar, se pudo llegar a la conclusión de cuales columnas serán utiles y cuales no, basados en esto, podemos proceder a eliminar las columnas que NO van a ser de algún uso para este análisis.

In [7]:
games = games.drop(columns=['publisher', 'title', 'url', 'reviews_url','specs','early_access'])

In [8]:
reviews = reviews.drop(columns=['user_url', 'funny', 'posted', 'last_edited', 'helpful'])

In [9]:
items = items.drop(columns=['items_count', 'steam_id', 'user_url'])

### Valores nulos

El análisis se realizó con df.isnull().sum() 
- El Dataframe Games contiene 88310 filas con todas sus celdas en Nan, todas estas serán eliminadas del Dataframe; los valores faltantes de las filas restantes las cuales contienen información serán tratadas en las siguientes secciones.
- El Dataframe Reviews contiene 4 columnas y 28 filas con valores nulos en 3 de sus columnas, estas serán eliminadas ya que no nos brindan ninguna información.
- El Dataframe Items contiene 16806 filas con 4 de 5 columnas en Nan, estás serán eliminadas ya que no contienen ninguna información valiosa.


In [10]:
games = games.dropna(how='all')

In [11]:
reviews = reviews.dropna(thresh=3)

In [12]:
items = items.dropna()

### Valores faltantes y Tipo de dato

El Dataframe Games contiene valores que deben ser normalizados, corregidos y rellenados. A continuación todos los procedimientos que se le realizaron:

1. La columna Price contiene muchas celdas en string, las cuales en su mayoría deberían ser 0.00, ya que aluden a juegos que son 'Free To Play'. A excepción de 2 precios los cuales si contienen un valor especifico (499.00 y 449.00), estos serán corregidos por su valor numerico correspondiente y el resto de strings serán convertidos en 0.00. Ya teniendo todos los valores de la columna Price en tipo numerico puede ser modificado su tipo de dato.

In [13]:
games['price'] = games['price'].replace('Starting at $499.00', 499.00)
games['price'] = games['price'].replace('Starting at $449.00', 449.00)

mask = games['price'].apply(lambda x: isinstance(x, str))
games.loc[mask, 'price'] = 0.00

# Se le agrega el errors para las celdas que contienen None
games['price'] = pd.to_numeric(games['price'], errors='coerce')

2. La columna Release_date contiene la fecha de lanzamiento de los juegos, vienen muchos formatos dentro de esta que deberán ser normalizados, meses en texto, estaciones del año...Se construyo entonces una función para todos los casos detectados y poder convertir esta columna en tipo de dato fecha, para posteriormente extraer el año (Se necesita para las consultas) y cambiar el nombre de la columna por 'year'.

In [14]:
def convertir_fecha(fecha):
    # Formatos Mes año, mes año, año...
    for fmt in ['%B %Y', '%b %Y', '%Y-%m-%d', '%Y']:
        try:
            return pd.to_datetime(fecha, format=fmt)
        except ValueError:
            continue

    # Casos donde hay texto antes del año
    match = re.search(r'\b(\d{4})\b', fecha)
    if match:
        # Si encuentra un año, convertirlo
        return pd.to_datetime(match.group(1), format='%Y')

    return pd.NaT  # Si no coincide con ningún formato, devuelve NaT

# Aplicar la función de conversión
games['release_date'] = games['release_date'].apply(convertir_fecha)

# Extraer el año
games['release_date'] = games['release_date'].dt.year

# Cambio de nombre de la columna release_date
games = games.rename(columns={'release_date': 'year'})

3. Las columnas 'app_name' e 'id' serán renombradas para que coincidan con los nombres de las columnas de las tablas Reviews e Items, esto con el fin de poder hacer un 'merged' o 'join' en las consultas posteriores.

In [15]:
games = games.rename(columns={'app_name': 'item_name'})

games = games.rename(columns={'id': 'item_id'})

4. Se necesita hacer el cambio de tipo de dato a dos columnas: 'item_id' y 'year', para las consultas posteriores...pero esto requiere que no existan valores en None, por ende se tomó la decisión de convertirlos en 0.00 para posteriormente hacer el cambio.

In [16]:
games['item_id'] = games['item_id'].fillna(0)
games['item_id'] = games['item_id'].astype(int)

games['year'] = games['year'].fillna(0)
games['year'] = games['year'].astype(int)

El Dataframe Items, necesita cambiar el tipo de dato de item_id para poder hacer merged.

In [17]:
items['item_id'] = items['item_id'].astype(int)

### Valores duplicados

Después de tener los Dataframes ya limpios, se procede por último a borrar las filas exactamente iguales. Ahora los Datasets están listos para ser trabajados.

In [18]:
games = games.drop_duplicates(subset=['item_name', 'year', 'price', 'item_id', 'developer'])

In [19]:
reviews = reviews.drop_duplicates()

In [20]:
items = items.drop_duplicates()

### Análisis de Sentimiento con NLP

La función realizada aquí se centra en analizar el sentimiento sobre cada fila, basándose en dos campos: review (reseña) y recommend (si el usuario recomienda o no el juego).Si la reseña está vacía, utiliza la recomendación del usuario para determinar el sentimiento (positivo o negativo). Si la reseña está presente, usa la polaridad del texto con TextBlob para clasificar la reseña como positiva, negativa o neutral, y guarda el resultado en la columna de review.

In [21]:
def analizar_sentimiento(row):
    reseña = row['review']
    recomendacion = row['recommend']
    
    # Caso: la reseña está vacía
    if pd.isnull(reseña) or reseña.strip() == '':
        # Si no hay reseña, usar la recomendación para determinar el sentimiento
        if recomendacion:
            return 2  # Si recomienda el juego, asignar positivo
        else:
            return 0  # Si no recomienda el juego, asignar negativo
    
    # Si la reseña está presente, hacer el análisis de sentimiento
    analysis = TextBlob(reseña).sentiment.polarity
    
    # Asignar el sentimiento basado en la polaridad
    if analysis < -0.1:
        return 0  # Sentimiento negativo
    elif analysis > 0.1:
        return 2  # Sentimiento positivo
    else:
        return 1  # Neutral

# Aplicar la función al dataset
reviews['review'] = reviews.apply(analizar_sentimiento, axis=1)

### Guardar los archivos

Se instala la librería pyarrow y se guardan los archivos ya tratados y limpios en formato tipo Parquet.

In [22]:
# Ruta especifica donde quiero guardar los archivos parquet
ruta_games = 'C:\\Users\\Sarita\\Desktop\\SOY HENRY\\Proyecto 1\\games_cleaned.parquet'
ruta_reviews = 'C:\\Users\\Sarita\\Desktop\\SOY HENRY\\Proyecto 1\\reviews_cleaned.parquet'
ruta_items = 'C:\\Users\\Sarita\\Desktop\\SOY HENRY\\Proyecto 1\\items_cleaned.parquet'

# Guarda el DataFrame en formato parquet en la ruta especificada
games.to_parquet(ruta_games, index=False)
reviews.to_parquet(ruta_reviews, index=False)
items.to_parquet(ruta_items, index=False)

### Archivos Render

Ya que es una aplicación gratuita de poca memoria disponible y se necesitan probar las consultas realizadas, se hará entonces con una muestra representativa de los Dataframes.

In [31]:
# Define el tamaño de la muestra (5% de las filas)
sample_fraction = 0.05

# Cargar los DataFrames completos
games_sample = games.sample(frac=sample_fraction, random_state=42)
reviews_sample = reviews.sample(frac=sample_fraction, random_state=42)
items_sample = items.sample(frac=sample_fraction, random_state=42)

# Guardar las muestras en archivos Parquet
games_sample.to_parquet('games_sample.parquet')
reviews_sample.to_parquet('reviews_sample.parquet')
items_sample.to_parquet('items_sample.parquet')


In [37]:
games_sample.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1607 entries, 93213 to 110765
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   genres     1457 non-null   object 
 1   item_name  1607 non-null   object 
 2   year       1607 non-null   int64  
 3   tags       1601 non-null   object 
 4   price      1543 non-null   float64
 5   item_id    1607 non-null   int64  
 6   developer  1448 non-null   object 
dtypes: float64(1), int64(2), object(4)
memory usage: 100.4+ KB


In [38]:
items_sample.info()

<class 'pandas.core.frame.DataFrame'>
Index: 254705 entries, 1784637 to 3695348
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           254705 non-null  object 
 1   item_id           254705 non-null  int64  
 2   item_name         254705 non-null  object 
 3   playtime_forever  254705 non-null  float64
 4   playtime_2weeks   254705 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 11.7+ MB


In [39]:
reviews_sample.info()


<class 'pandas.core.frame.DataFrame'>
Index: 2922 entries, 11926 to 8576
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    2922 non-null   object
 1   item_id    2922 non-null   object
 2   recommend  2922 non-null   object
 3   review     2922 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 114.1+ KB


In [40]:
games_sample.columns = games_sample.columns.str.strip()
items_sample.columns = items_sample.columns.str.strip()


In [41]:
games_sample = games_sample.drop_duplicates(subset="item_name")
items_sample = items_sample.drop_duplicates(subset="item_name")

In [42]:
games_sample = games_sample.dropna(subset=["item_name"])
items_sample = items_sample.dropna(subset=["item_name"])
